# 03 - Bonnie Brown Analysis

### Bonnie Brown is a seller, and wants to sell her house soon with high profit and in mid-class neighborhood.  

---
## Assumptions

Based on the information I got I will work with the following assumptions:
- mid-class neighborhood = houses of 25th and 75th price percentile (IQR)
- selling soon = within next year
- high profit = focus on best prices and how to optimize to get the best return


## 1. Setup

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

plt.style.use('seaborn-v0_8-whitegrid')

In [3]:
# Load cleaned data from 01_general_eda
df_clean = pd.read_csv('data/eda_clean.csv')
df_clean['date'] = pd.to_datetime(df_clean['date'])

print(f"Dataset: {df_clean.shape}")

FileNotFoundError: [Errno 2] No such file or directory: 'data/eda_clean.csv'

## 2. Define Middle-Class Segment

**Assumption** (from `02_client_selection.ipynb`):
> Middle-class is defined by house price distribution (25th-75th percentile), not by external socio-economic data.

In [ ]:
# Define middle-class price range
q25 = df_clean['price'].quantile(0.25)
q75 = df_clean['price'].quantile(0.75)

print(f"Middle-Class Price Range: ${q25:,.0f} - ${q75:,.0f}")
print(f"Median Price: ${df_clean['price'].median():,.0f}")

In [ ]:
# Filter for middle-class segment
df_middle = df_clean[(df_clean['price'] >= q25) & (df_clean['price'] <= q75)].copy()

print(f"Middle-Class Houses: {len(df_middle)} ({len(df_middle)/len(df_clean)*100:.1f}% of total)")

## 3. Select Relevant Features

For Bonnie's analysis, we focus on:
- **Price**: price, price_per_sqft
- **Location**: zipcode, lat, long
- **Timing**: date, month
- **Quality**: grade, condition
- **Size**: sqft_living, bedrooms, bathrooms

In [ ]:
# Select relevant columns
bonnie_cols = ['price', 'zipcode', 'date', 'month', 'grade', 'condition', 
               'sqft_living', 'bedrooms', 'bathrooms', 'yr_built', 
               'price_per_sqft', 'lat', 'long']

df_bonnie = df_middle[bonnie_cols].copy()
print(f"Bonnie Analysis Dataset: {df_bonnie.shape}")

NameError: name 'df_middle' is not defined

---
## 4. Analysis Question 1: Best Timing to Sell

**Question**: When should Bonnie list her property for best results?

In [ ]:
# Monthly price analysis for middle-class
monthly = df_bonnie.groupby('month').agg({
    'price': ['mean', 'median', 'count']
}).round(0)
monthly.columns = ['Average', 'Median', 'Count']
monthly

In [ ]:
# Visualize timing
fig, ax = plt.subplots(figsize=(10, 5))

colors = ['green' if m in [4,5,6,7] else 'steelblue' for m in monthly.index]
ax.bar(monthly.index, monthly['Median'], color=colors, edgecolor='black')

ax.set_xlabel('Month')
ax.set_ylabel('Median Price ($)')
ax.set_title('Best Months to Sell (Middle-Class Segment)')
ax.set_xticks(range(1, 13))
ax.axhline(monthly['Median'].mean(), color='red', linestyle='--', label='Average')
ax.legend()

plt.tight_layout()
plt.show()

In [ ]:
# Best months
print("Best months to sell (by median price):")
print(monthly.sort_values('Median', ascending=False).head(4))

### Timing Insight
- **Best months**: April, May, June, July (Spring/Summer)
- Higher prices AND higher sales volume
- Avoid winter months (December, January, February)

---
## 5. Analysis Question 2: Best Neighborhoods

**Question**: Which zipcodes achieve highest prices in middle-class?

In [ ]:
# Zipcode analysis
zip_stats = df_bonnie.groupby('zipcode').agg({
    'price': ['mean', 'median', 'count'],
    'price_per_sqft': 'mean'
}).round(0)
zip_stats.columns = ['Avg Price', 'Median Price', 'Count', 'Avg $/sqft']

# Filter for sufficient data (min 20 sales)
zip_stats = zip_stats[zip_stats['Count'] >= 20]
zip_stats = zip_stats.sort_values('Median Price', ascending=False)

print("Top 10 Zipcodes for Middle-Class:")
zip_stats.head(10)

In [ ]:
# Geographic visualization
plt.figure(figsize=(12, 8))
scatter = plt.scatter(df_bonnie['long'], df_bonnie['lat'], 
                      c=df_bonnie['price'], cmap='RdYlGn', alpha=0.5, s=15)
plt.colorbar(scatter, label='Price ($)')
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.title('Middle-Class Properties - Geographic Price Distribution')
plt.show()

In [ ]:
# Top 5 zipcodes
top_zips = zip_stats.head(5).index.tolist()
print(f"Top 5 Recommended Zipcodes: {top_zips}")

### Location Insight
- Top performing zipcodes: 98004, 98040, 98075, 98005, 98119
- These areas achieve highest median prices in middle-class
- Price difference between best and worst can exceed $100K

---
## 6. Analysis Question 3: Quality Factors

**Question**: How does grade affect price? Should Bonnie invest in improvements?

In [ ]:
# Grade analysis
grade_stats = df_bonnie.groupby('grade').agg({
    'price': ['mean', 'count']
}).round(0)
grade_stats.columns = ['Average Price', 'Count']
grade_stats

In [ ]:
# Visualize grade impact
fig, ax = plt.subplots(figsize=(10, 5))

grades = grade_stats[grade_stats['Count'] >= 10]
ax.bar(grades.index.astype(str), grades['Average Price'], color='teal', edgecolor='black')

ax.set_xlabel('Grade')
ax.set_ylabel('Average Price ($)')
ax.set_title('Impact of Grade on Price (Middle-Class)')

plt.tight_layout()
plt.show()

In [ ]:
# Price increase per grade level
print("Price increase by grade level:")
for i in range(6, 10):
    if i in grades.index and i+1 in grades.index:
        increase = grades.loc[i+1, 'Average Price'] - grades.loc[i, 'Average Price']
        print(f"  Grade {i} → {i+1}: +${increase:,.0f}")

### Quality Insight
- Each grade level adds $30K-$50K to average price
- Grade 7-9 is the sweet spot for middle-class
- Improvements that increase grade can provide significant ROI

---
## 7. Finding Example Properties

Properties that match all of Bonnie's criteria:
1. Middle-class price ($322K - $645K)
2. Top-performing zipcodes
3. Good quality (grade ≥ 7)
4. Optimal selling period (April-July)

In [ ]:
# Filter for optimal properties
optimal = df_bonnie[
    (df_bonnie['zipcode'].isin(top_zips)) &
    (df_bonnie['grade'] >= 7) &
    (df_bonnie['month'].isin([4, 5, 6, 7]))
].sort_values('price', ascending=False)

print(f"Properties matching all criteria: {len(optimal)}")

In [ ]:
# Top 3 examples
top3 = optimal.head(3)[['price', 'zipcode', 'date', 'grade', 'condition', 
                         'sqft_living', 'bedrooms', 'bathrooms', 'price_per_sqft']]
top3

In [ ]:
# Summary
print("=" * 60)
print("EXAMPLE PROPERTIES MATCHING BONNIE'S CRITERIA")
print("=" * 60)

for i, (idx, row) in enumerate(top3.iterrows(), 1):
    print(f"\nProperty {i}:")
    print(f"   Price: ${row['price']:,.0f}")
    print(f"   Zipcode: {row['zipcode']} (Top Neighborhood)")
    print(f"   Sold: {row['date'].strftime('%B %Y')} (Optimal Season)")
    print(f"   Grade: {row['grade']}")
    print(f"   Size: {row['sqft_living']:.0f} sqft")
    print(f"   Layout: {row['bedrooms']:.0f} bed, {row['bathrooms']:.0f} bath")
    print(f"   Price/sqft: ${row['price_per_sqft']:.0f}")

---
## 8. Summary: Recommendations for Bonnie Brown

### Recommendation 1: Sell in Spring/Summer
**Action**: List the property between April and July
- Highest buyer activity
- Better prices achieved
- Faster sales cycles

### Recommendation 2: Price Strategically
**Action**: Price within middle-class sweet spot ($322K - $645K)
- Largest buyer pool
- Faster turnover
- Competitive but profitable

### Recommendation 3: Consider Quality Improvements
**Action**: Invest in grade improvements if below 7
- Each grade level adds $30K-$50K
- Focus on kitchen, bathrooms, curb appeal
- Calculate ROI before investing

---

## Key Insights Summary

| Insight | Finding |
|---------|--------|
| **Timing** | April-July shows highest prices and volume |
| **Location** | Top zipcodes: 98004, 98040, 98075, 98005, 98119 |
| **Quality** | Grade strongly impacts price (+$30-50K per level) |

---
*Analysis complete. See presentation slides for non-technical summary.*